# Further Pre-training MobileBERT MLM with Client-side Adam (Stackoverflow)

In [ ]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Google Colab settings

In [ ]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    # Mount Google Drive root directory
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/Colab Notebooks/BERTerated'
    
    # List the directory contents
    !ls

### CUDA Multi GPU

In [ ]:
# Use this code snippet to use specific GPUs
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

In [ ]:
# IPython reloading magic
%load_ext autoreload
%autoreload 2

In [ ]:
# Install required packages
# !pip install -r requirements.txt

## Import packages

In [ ]:
import tensorflow as tf

In [ ]:
tf_physical_devices_gpu = tf.config.list_physical_devices('GPU')

# Allow the growth of GPU memory consumption to take place incrementally
if tf_physical_devices_gpu:
    for gpu in tf_physical_devices_gpu:
        tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import os
import sys
import random
import datetime
import json
import pathlib
import itertools
import time

import numpy as np
import tensorflow_federated as tff
import tensorflow_text as tf_text
import tensorflow_addons as tfa
import transformers

import nest_asyncio
nest_asyncio.apply()

import fedavg
import fedavg_client
import datasets
import utils


# Random seed settings
random_seed = 692
random.seed(random_seed) # Python
np.random.seed(random_seed) # NumPy
tf.random.set_seed(random_seed) # TensorFlow

# Test if TFF is working
tff.federated_computation(lambda: 'Hello, World!')()

In [ ]:
# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("TensorFlow version: " + tf.__version__)
print("TensorFlow Federated version: " + tff.__version__)
print("Transformers version: " + transformers.__version__)

In [ ]:
!nvidia-smi

In [ ]:
tf_logical_devices_cpu = tf.config.list_logical_devices('CPU')
tf_logical_devices_gpu = tf.config.list_logical_devices('GPU')

## Experiment Settings

In [ ]:
EXPERIMENT_CONFIG = {}

EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'] = 'google/mobilebert-uncased'
EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'] = os.path.join('.', 'transformers_cache')

EXPERIMENT_CONFIG['TOTAL_ROUNDS'] = 50 # Number of total training rounds
EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] = 1 # How often to evaluate

EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'] = 10 # How many clients to sample per round.
EXPERIMENT_CONFIG['TEST_CLIENTS_PER_ROUND'] = 10
EXPERIMENT_CONFIG['CLIENT_EPOCHS_PER_ROUND'] = 3

EXPERIMENT_CONFIG['BATCH_SIZE'] = 16 # Batch size used on the client.
EXPERIMENT_CONFIG['TEST_BATCH_SIZE'] = 8 # Minibatch size of test data.

# Maximum length of input token sequence for BERT.
EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'] = 128

# Optimizer configuration
EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'] = 1.0 # Server learning rate.
EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'] = 5e-5 # Client learning rate

# Client dataset setting
EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] = 2000
EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT'] = 500

# Path to save trained weights and logs
EXPERIMENT_CONFIG['RESULTS_DIRECTORY'] = os.path.join(
    '.', 'results',
    'mobilebert_mlm_stackoverflow_fedclientadam',
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)

EXPERIMENT_CONFIG['RESULTS_LOG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "logs")
EXPERIMENT_CONFIG['RESULTS_MODEL'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "model")
EXPERIMENT_CONFIG['RESULTS_CONFIG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "config")

In [ ]:
# Dump all the configuration into a json file
pathlib.Path(EXPERIMENT_CONFIG['RESULTS_CONFIG']).mkdir(parents=True, exist_ok=True)

with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "config.json"), 'w') as config_file:
    json.dump(EXPERIMENT_CONFIG, config_file, indent=6)

In [ ]:
# TFF executor factory settings
# Reference: https://www.tensorflow.org/federated/api_docs/python/tff/backends/native/set_local_execution_context
tff.backends.native.set_local_execution_context(
    num_clients=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
    max_fanout=100,
    clients_per_thread=1,
    server_tf_device=tf_logical_devices_cpu[0],
    client_tf_devices=tf_logical_devices_cpu,
)

## Dataset

### Dataset loader

In [ ]:
train_client_data = tff.python.simulation.hdf5_client_data.HDF5ClientData(
    os.path.join('.', 'tff_cache', 'datasets', 'stackoverflow_train.h5'))
# held_out_client_data = tff.python.simulation.hdf5_client_data.HDF5ClientData(
#     os.path.join('.', 'tff_cache', 'datasets', 'stackoverflow_held_out.h5'))
test_client_data = tff.python.simulation.hdf5_client_data.HDF5ClientData(
    os.path.join('.', 'tff_cache', 'datasets', 'stackoverflow_test.h5'))

### Tokenizer

In [ ]:
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

In [ ]:
# Imitate transformers tokenizer with TF.Text Tokenizer
tokenizer_tf_text, vocab_lookup_table, special_ids_mask_table = \
datasets.preprocessing_for_bert.convert_huggingface_tokenizer(bert_tokenizer)

### Preprocessing

In [ ]:
def check_empty_snippet(x):
    return tf.strings.length(x['tokens']) > 0

def tokenizer_and_mask_wrapped(x):

    masked, labels = datasets.preprocessing_for_bert.tokenize_and_mask(tf.reshape(x['tokens'], shape=[1]),
                                                                       max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
                                                                       bert_tokenizer_tf_text=tokenizer_tf_text,
                                                                       vocab_lookup_table=vocab_lookup_table,
                                                                       special_ids_mask_table=special_ids_mask_table,
                                                                       cls_token_id=bert_tokenizer.cls_token_id,
                                                                       sep_token_id=bert_tokenizer.sep_token_id,
                                                                       pad_token_id=bert_tokenizer.pad_token_id,
                                                                       mask_token_id=bert_tokenizer.mask_token_id)

    return (masked, labels)

def preprocess_for_train(train_dataset):
    return (
        train_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly BATCH_SIZE
        # and make the shape **exactly** (BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['BATCH_SIZE'])
        # Repeat to make each client train multiple epochs
        .repeat(count=EXPERIMENT_CONFIG['CLIENT_EPOCHS_PER_ROUND'])
    )
    
def preprocess_for_test(test_dataset):
    return (
        test_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly TEST_BATCH_SIZE
        # and make the shape **exactly** (TEST_BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['TEST_BATCH_SIZE'])
    )

### Training set

In [ ]:
train_client_data = train_client_data.preprocess(preprocess_fn=lambda x: x.filter(check_empty_snippet))

In [ ]:
train_client_data = train_client_data.preprocess(preprocess_fn=preprocess_for_train)

In [ ]:
print(train_client_data.element_type_structure)

In [ ]:
# Since the dataset is pretty large, we randomly select TRAIN_NUM_CLIENT_LIMIT number of clients.
all_train_client_ids = train_client_data.client_ids

random.shuffle(all_train_client_ids)

if EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] > 0:
    selected_train_client_ids = all_train_client_ids[0:EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT']]
else:
    selected_train_client_ids = all_train_client_ids

In [ ]:
# Save the list of selected training clients
with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "train_clients_list.json"), 'w') as train_clients_file:
    json.dump(selected_train_client_ids, train_clients_file, indent=6)

In [ ]:
train_client_states = {}

# Initialize client states for all clients (selected for the entire simulation)
for i, client_id in enumerate(selected_train_client_ids):
    train_client_states[client_id] = fedavg_client.ClientState(client_serial=i, num_processed=0)

### Test set

In [ ]:
test_client_data = test_client_data.preprocess(preprocess_fn=lambda x: x.filter(check_empty_snippet))

In [ ]:
test_client_data = test_client_data.preprocess(preprocess_fn=preprocess_for_test)

In [ ]:
print(test_client_data.element_type_structure)

In [ ]:
# Since the dataset is pretty large, we randomly select TEST_NUM_CLIENT_LIMIT number of clients.
all_test_client_ids = test_client_data.client_ids

random.shuffle(all_test_client_ids)

if EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT'] > 0:
    selected_test_client_ids = all_test_client_ids[0:EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT']]
else:
    selected_test_client_ids = all_test_client_ids

In [ ]:
# Save the list of selected training clients
with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "test_clients_list.json"), 'w') as test_clients_file:
    json.dump(selected_test_client_ids, test_clients_file, indent=6)

## Model

In [ ]:
bert_model = transformers.TFAutoModelForPreTraining.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

In [ ]:
print(bert_model.config)

In [ ]:
# Due to the limitations with Keras subclasses, we can only use the main layer part from pretrained models
# and add output heads by ourselves
bert_keras_converted = utils.convert_huggingface_mlm_to_keras(
    huggingface_model=bert_model,
    max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
)

In [ ]:
# Use lists of NumPy arrays to backup pretained weights
bert_pretrained_trainable_weights = []
bert_pretrained_non_trainable_weights = []

for w in bert_keras_converted.trainable_weights:
    bert_pretrained_trainable_weights.append(w.numpy())

for w in bert_keras_converted.non_trainable_weights:
    bert_pretrained_non_trainable_weights.append(w.numpy())

In [ ]:
def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""

    loss = utils.MaskedLMCrossEntropy()

    model_wrapped = utils.KerasModelWrapper(
        tf.keras.models.clone_model(bert_keras_converted),
        train_client_data.element_type_structure, loss)

    return model_wrapped

## Training

### Training setups

In [ ]:
summary_writer = tf.summary.create_file_writer(EXPERIMENT_CONFIG['RESULTS_LOG'])

In [ ]:
def server_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'])

def client_optimizer_fn():
    return transformers.AdamWeightDecay(
        learning_rate=EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'],
        weight_decay_rate=0.01,
    )

In [ ]:
%%time

iterative_process = fedavg.build_federated_averaging_process(
    model_fn=tff_model_fn,
    model_input_spec=train_client_data.element_type_structure,
    initial_trainable_weights=bert_pretrained_trainable_weights,
    initial_non_trainable_weights=bert_pretrained_non_trainable_weights,
    server_optimizer_fn=server_optimizer_fn, 
    client_optimizer_fn=client_optimizer_fn)

In [ ]:
%%time

server_state = iterative_process.initialize()

In [ ]:
metric_eval = tfa.metrics.MeanMetricWrapper(fn=utils.calculate_masked_lm_cross_entropy, name='ce')

In [ ]:
# The model for calculating validation loss only
# (This happens outside FedAvg)
model_final = utils.KerasModelWrapper(
    tf.keras.models.clone_model(bert_keras_converted),
    train_client_data.element_type_structure,
    utils.MaskedLMCrossEntropy(),
    tf_device_identifier="/GPU:0")

### Training loop

In [ ]:
%%time

with summary_writer.as_default():
    for round_num in range(1, EXPERIMENT_CONFIG['TOTAL_ROUNDS'] + 1):        
        # FedAvg
        print(f'Round {round_num} start!')

        # Training client selection
        sampled_client_serials = np.random.choice(
            len(selected_train_client_ids),
            size=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
            replace=False)

        sampled_train_data = [
            train_client_data.create_tf_dataset_for_client(selected_train_client_ids[client_serial])
            for client_serial in sampled_client_serials
        ]
        
        sampled_client_states = [
            train_client_states[selected_train_client_ids[client_serial]]
            for client_serial in sampled_client_serials
        ]
        
        print("Selected client serials:", sampled_client_serials)

        current_round_start_time = time.time()
        
        server_state, new_client_states, train_loss = iterative_process.next(
            server_state, sampled_client_states, sampled_train_data)
        
        current_round_end_time = time.time()
        
        currnt_round_running_time = current_round_end_time - current_round_start_time

        print(f'Round {round_num} training loss: {train_loss}')
        print(f'Round {round_num} execution time: {currnt_round_running_time}')
        
        # Record the current round's training loss to the log
        tf.summary.scalar('train_loss', train_loss, step=round_num)
        tf.summary.scalar('train_running_time', currnt_round_running_time, step=round_num)
        
        print()
        
        # Update client states
        print("Updating client states.")

        for state in new_client_states:
            train_client_states[selected_train_client_ids[state.client_serial]] = state

        print()
        
        print("Recording client statistics:")
        
        for client_id in selected_train_client_ids:
            state = train_client_states[client_id]
            
            tf.summary.scalar(
                'client_' + str(int(state.client_serial)) + '_num_processed',
                int(state.num_processed), step=round_num)

        print()

        # Evaluation
        if round_num % EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] == 0:
            model_final.from_weights(server_state.model_weights)

            # Test dataset generation for this round
            print("Calculating validation metric:")

            # Testing client selection
            sampled_test_client_ids = np.random.choice(
                selected_test_client_ids,
                size=EXPERIMENT_CONFIG['TEST_CLIENTS_PER_ROUND'],
                replace=False)
            
            print("Test clients:", sampled_test_client_ids)

            sampled_test_data = [
                test_client_data.create_tf_dataset_for_client(test_client_id)
                for test_client_id in sampled_test_client_ids
            ]
            
            sampled_test_data_all_merged = sampled_test_data[0]

            if len(sampled_test_data) > 1:
                for d in range(1, len(sampled_test_data)):
                    sampled_test_data_all_merged = sampled_test_data_all_merged.concatenate(sampled_test_data[d])

            current_round_validation_start_time = time.time()
            
            current_round_validation_metric = utils.keras_evaluate(
                model_final.keras_model, sampled_test_data_all_merged, metric_eval, "/GPU:0")
            
            current_round_validation_end_time = time.time()
            
            current_round_validation_runnning_time = current_round_validation_end_time - current_round_validation_start_time

            print(f'Round {round_num} validation metric: {current_round_validation_metric}')
            print(f'Round {round_num} validation time: {current_round_validation_runnning_time}')
            
            # Write down train_metrics to the log
            tf.summary.scalar('validation_metric', current_round_validation_metric, step=round_num)
            tf.summary.scalar('validation_running_time', current_round_validation_runnning_time, step=round_num)
            
        print()

### Save the trained model

In [ ]:
model_final.keras_model.save(EXPERIMENT_CONFIG['RESULTS_MODEL'])